<a href="https://colab.research.google.com/github/AndresMontesDeOca/TextMining/blob/main/Text_Mining_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

#### Universidad Austral<br>
#### Maestría en Minería de Datos y Gestión del Conocimiento<br>
#### Text Mining<br>
#### CGC: Clasificador de Géneros Cinematográficos<br>


#### Integrantes:<br>
Alejandra Reyes<br>
Andres Montes de Oca<br>
Rafael Gimenez<br>
Soledad Ríos<br>
Tomas Sauro

</center>

## Introducción
Título: Clasificador de Géneros Cinematográficos (CGC)

Nota: utilizar mas de un modelo (x modelos) y que el numero sea impar para usar criterio democratico

Problemática: En un servicio que releva los contenidos disponibles en plataformas streaming, con frecuencuencia se detectan contenidos sin información de género pero se cuenta con la sinopsis del contenido.

Nombre del servicio y empresa con la problematica: [Content Pulse | BB Media](https://bb.vision/content-pulse-en/)

Objetivo: Entrenar un modelo que dada una sinopsis sobre una película retorne los géneros del contenido.

Hoja de Ruta:

1. Acceso a los datos
2. Tabulación de los datos
3. Limpieza de datos
4. Exploración de modelos
5. Pruebas
6. Analísis de resultados
7. Conclusiones

Posibles contratiempos:

1. Datos faltantes, incompletos, no estandarizados.
2. Fallar en una buena separacion de los subdataset para testing y validation (ejemplo, quedarnos con sinopsis para testing muy ricas en descripción y para validation no).

## Librerias

In [ ]:
# Verificación e instalación
import importlib
import subprocess

def instalar_librerias(packages):
    [importlib.import_module(package) if package in locals() else subprocess.call(['pip', 'install', package]) for package in packages]

# Lista de librerías a verificar e instalar
listado_librerias = ['requests', 'json', 'datetime', 'gdown', 'numpy', 'pandas', 'seaborn', 'matplotlib', 'tabulate', 'scikit-learn']

# Verificar e instalar las librerías
instalar_librerias(listado_librerias)

In [ ]:
# Importación
import requests
import json
import datetime
import gdown

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate

from sklearn.model_selection import train_test_split

# Ignorar Warnings
import warnings
warnings.filterwarnings("ignore")

## 0. Acceso a los datos (Ejemplo con algunas variables usando la API de TMDB)


Se utiliza una base de datos colaborativas con acceso abierto llamada TMDB obtenido los datos mediante su [API](https://developer.themoviedb.org/docs)

In [ ]:
# Credencial
api_key = 'bf0a945ba271caf72a6a1b1f53a1084d'

# URL base
base_url = 'https://api.themoviedb.org/3/'

# Endpoint para obtener la lista de películas y series de tv
movie_endpoint = 'discover/movie'
tv_endpoint = 'discover/tv'

# Parámetros
params = {
    'api_key': api_key,
    'sort_by': 'popularity.desc' # Se usa este orden para verificar si los datos obtenidos son correctos (los contenidos mas populares son mas conocidos siendo mas facil validar)
}

# Función para obtener nombres de actores
def get_cast_names(content_type, content_id):
    credits_endpoint = f'{content_type}/{content_id}/credits'
    credits_params = {
        'api_key': api_key
   }
    credits_response = requests.get(base_url + credits_endpoint, params=credits_params)
    credits_data = credits_response.json()
    cast_names = [actor['name'] for actor in credits_data['cast'][:3]] # Dado que los actores estan ordenados por importancia, nos quedamos con los 3 primeros (protagonistas)
    return ', '.join(cast_names)

# Función para obtener nombres de compañías de producción
def get_production_companies(content_type, content_id):
    details_endpoint = f'{content_type}/{content_id}'
    details_params = {
        'api_key': api_key
    }
    details_response = requests.get(base_url + details_endpoint, params=details_params)
    details_data = details_response.json()
    production_companies = ', '.join([company['name'] for company in details_data.get('production_companies', [])])
    return production_companies

# Realizar solicitudes GET a la API y obtener datos para películas
movie_response = requests.get(base_url + movie_endpoint, params=params)
movie_data = movie_response.json()

# Realizar solicitudes GET a la API y obtener datos para programas de televisión
tv_response = requests.get(base_url + tv_endpoint, params=params)
tv_data = tv_response.json()

# Procesar los resultados de películas y programas de televisión
results_list = []

for content_type, content_data in [('movie', movie_data), ('tv', tv_data)]:
    for result in content_data['results']:
        content_id = result['id']
        title = result['title'] if content_type == 'movie' else result['name']
        release_date = result['release_date'] if content_type == 'movie' else result['first_air_date']
        overview = result['overview']
        poster_path = result['poster_path']
        popularity = result['popularity']
        vote_count = result['vote_count']
        vote_average = result['vote_average']

        # Obtener nombres de actores utilizando la función
        cast_names = get_cast_names(content_type, content_id)

        # Obtener nombres de compañías de producción utilizando la nueva función
        production_companies = get_production_companies(content_type, content_id)

        results_list.append({
            'type': content_type.capitalize(),
            'id': content_id,
            'title': title,
            'release': release_date,
            'synopsis': overview,
            'cast': cast_names,
            'productions': production_companies,
            'popularity': popularity,
            'votes': vote_count,
            'score': vote_average,
            'imagen': f'https://image.tmdb.org/t/p/w500{poster_path}'
        })

# DataFrame
tmdb_base_20 = pd.DataFrame(results_list)
tmdb_base_20.head()

## 1. Acceso a los datos (Archivo preexistente con datos públicos previamente descargados desde TMDB)

In [ ]:
# URL del archivo Excel
url = "https://onedrive.live.com/download?resid=B5CCCD69939F6AA3%21985&authkey=!AMd0773xIAnUL_I&em=x&app=Excel"

# Descarga del archivo en el entorno de colab
output = "/content/file.xlsx"  # Ruta donde se guardará el archivo en Colab
gdown.download(url, output, quiet=False)

In [ ]:
# DataFrame
contents = pd.read_excel(output)
contents.head()

## 2. Exploración Inicial



In [ ]:
print('contents dataframe:', contents.shape)

In [ ]:
print(contents.dtypes)

In [ ]:
# Verificar si el campo id contiene valores únicos
id_is_unique = contents['id'].nunique() == len(contents['id'])
if id_is_unique:
    print("El campo 'id' tiene valores únicos.")
else:
    print("El campo 'id' tiene valores repetidos.")

In [ ]:
# Obtener un id repetido para verificar los registros que lo contienen

# Construir tabla de frecuencia por 'id'
id_frequency_table = contents['id'].value_counts().reset_index()
id_frequency_table.columns = ['id', 'Frequency']

# Filtrar la tabla de frecuencia para encontrar el primer ID con una frecuencia mayor a uno
firts_id_duplicated = id_frequency_table[id_frequency_table['Frequency'] > 1]['id'].iloc[0]

# Mostrar los registros que contienen el primer ID repetido
contents[contents['id'] == firts_id_duplicated]

In [ ]:
# Verificar si el campo id contiene valores únicos para cada tipo de contenido
type_contents = ['Series', 'Movie']

for type_contents in type_contents:
    # Filtrar el DataFrame por el campo 'type' para el tipo de contenido actual
    contents_by_type = contents[contents['type'] == type_contents]

    # Verificar si el campo 'id' en el DataFrame filtrado contiene valores únicos
    id_is_unique = contents_by_type['id'].nunique() == len(contents_by_type['id'])

    if id_is_unique:
        print(f"El campo 'id' tiene valores únicos para el tipo de contenido '{type_contents}'.")
    else:
        print(f"El campo 'id' tiene valores repetidos para el tipo de contenido '{type_contents}'.")

####**Variables**

- **id**: Identificador único (por tipo de contenido) para cada registro
- **type**: Tipo de contenido (Movies/Series)
- **title**: Título del contenido
- **year**: Año de lanzamiento del contenido
- **duration**: Duración del contenido expresada en minutos
- **synopsis**: Reseña del contenido
- **genres**: Géneros de los contenidos (separados con "," cuando hay más de uno
- **cast**: Listado de actores (separados con ",")
- **directors**: director/directores del contenido (separados con ",")
- **url**: Direccion web del contenido en tmdb

## 3. Exploración y Transformación

In [ ]:
# Verificación
contents.info()

In [ ]:
# Especificar variables categóricas
var_categoricas = [col for col in contents.columns if col in ['type', 'genres']]

# Convertir a categóricas las varibales correspondientes
contents[var_categoricas] = contents[var_categoricas].astype('category')

# Verificación
contents.dtypes

In [ ]:
contents.isna().sum()

In [ ]:
# Eliminar los registros con valor nulo en el campo synopsis
contents.dropna(subset=['synopsis'], inplace=True)
contents.isna().sum()

In [ ]:
# Calcular el porcentaje de valores nulos para cada columna
porcentaje_nulos = (contents.isnull().sum() / len(contents)) * 100
print("Porcentaje de valores nulos por columna:")
print(porcentaje_nulos)

In [ ]:
contents['genres'].value_counts()

In [ ]:
# Crear una columna con el género principal
contents['principal_genre'] = contents['genres'].str.split(',').str[0]
contents.head()

In [ ]:
contents['principal_genre'].value_counts()

In [ ]:
# Crear la columna 'synopsis_length' que contiene la longitud de cada sinopsis
contents['synopsis_length'] = contents['synopsis'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
contents['synopsis_length'].describe()

In [ ]:
# Crear la columna 'synopsis_length_interval' que contiene los intervalos de la longitud de cada sinopsis

# Definir los límites de los intervalos
bins = [1, 100, 200, 300, float('inf')]

# Definir las etiquetas para cada intervalo
labels = ['Muy corta', 'Corta', 'Moderada', 'Larga']

# Crear el campo 'synopsis_length_interval' que asigna un intervalo a cada longitud de sinopsis
contents['synopsis_length_interval'] = pd.cut(contents['synopsis_length'], bins=bins, labels=labels, include_lowest=True)
contents.head()


## 4. Separación de dataframes

In [ ]:
# Excluir columnas con las que no se trabajara
contents= contents.drop(['id', 'title', 'genres', 'url'], axis=1)

In [ ]:
# Registros sin genres (el problema a resolver)
contents_target = contents[contents['principal_genre'].isnull()]

# Registros con genres para modelar
contents = contents.drop(contents_target.index)

In [ ]:
# Tabla de contingencia (%) entre principal_genre synopsis_length_interval en contents
pd.crosstab(contents['principal_genre'], contents['synopsis_length_interval'])

In [ ]:
# Dividir el DataFrame en 70% train y 30% test, estratificado por "principal_genre" y "synopsis_length_interval"
contents_train, contents_test = train_test_split(contents, test_size=0.3, stratify=contents[['principal_genre', 'synopsis_length_interval']], random_state=2023)

In [ ]:
# Tabla de contingencia (%) entre principal_genre synopsis_length_interval en contents_train
pd.crosstab(contents_train['principal_genre'], contents_train['synopsis_length_interval'], normalize='index') * 100

In [ ]:
# Tabla de contingencia (%) entre principal_genre synopsis_length_interval en contents_test
pd.crosstab(contents_test['principal_genre'], contents_test['synopsis_length_interval'], normalize='index') * 100

## 5. Baseline Model

## 6. Andy's Model

In [ ]:
# Sampleamos el dataset

data['synopsis'] = data['synopsis']str.lower() # convertimos todo a minusculas
data = contents[['synopsis', 'principal_genre']].sample(1000, random_state=99) # sampleamos a 1000 registros
display(data.head())
print('Cantidad de Generos distintos:', len(data.principal_genre.unique()))

### Pre Procesamiento

In [ ]:
# !pip install pattern

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from pattern.en import lemma
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

vect = TfidfVectorizer(analyzer     = "word",
                       decode_error = "ignore",
                       encoding     = "str",
                       lowercase    = True,
                       stop_words   = stopwords.words("english"),
                       strip_accents= "ascii",
                       tokenizer    = lambda texts: [lemma(t) for t in word_tokenize(texts) if (t not in stopwords.words('english'))]
                       )


texts = data.synopsis.values
data_Tfidf = vect.fit_transform(texts)

X = data_Tfidf
y = data['principal_genre']
print(X.shape)

# División de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


# Modelos
model1 = RandomForestClassifier()
model2 = GradientBoostingClassifier()
model3 = SVC()
model4 = LogisticRegression()

# Entrenamiento del modelo de clasificación multietiqueta (Regresión Logística Multietiqueta)
classifier = MultiOutputClassifier(model4)
classifier.fit(X_train, y_train.to_frame())

# Predicción
y_pred = classifier.predict(X_test)

# Evaluación del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy:.2f}')